In [23]:
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import pandas_ta
from utils import calculate_returns, compute_atr, compute_macd
import warnings
warnings.filterwarnings('ignore')

sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

master_ticker_data = sp500.copy()
master_ticker_data.rename(columns={"Symbol":"ticker"}, inplace = True)
master_ticker_data['ticker'] = master_ticker_data['ticker'].str.replace('.', '-')

sp500['Symbol'] = sp500['Symbol'].str.replace('.', '-')

symbols_list = sp500['Symbol'].unique().tolist()

end_date = '2023-12-31'

start_date = pd.to_datetime(end_date)-pd.DateOffset(365*10)

df = yf.download(tickers=symbols_list,
                 start=start_date,
                 end=end_date).stack()

df.index.names = ['date', 'ticker']

df.columns = df.columns.str.lower()

df

[*********************100%%**********************]  503 of 503 completed

2 Failed downloads:
['GEV', 'SOLV']: YFChartError("%ticker%: Data doesn't exist for startDate = 1388638800, endDate = 1703998800")


Price               adj close       close        high         low        open  \
date       ticker                                                               
2014-01-02 A        36.873859   40.207439   40.844063   40.164520   40.844063   
           AAL      23.907921   25.360001   25.820000   25.059999   25.070000   
           AAPL     17.273224   19.754642   19.893929   19.715000   19.845715   
           ABBV     33.865421   51.980000   52.330002   51.520000   52.119999   
           ABT      31.214533   38.230000   38.400002   38.000000   38.090000   
...                       ...         ...         ...         ...         ...   
2023-12-29 XYL     113.729851  114.360001  114.680000  113.930000  114.089996   
           YUM     129.373611  130.660004  131.250000  130.210007  130.410004   
           ZBH     121.471581  121.699997  122.400002  121.239998  121.459999   
           ZBRA    273.329987  273.329987  276.309998  272.769989  274.730011   
           ZTS     196.363464  197.369995  198.009995  196.250000  196.679993   

Price                   volume  
date       ticker               
2014-01-02 A         2678848.0  
           AAL       8997900.0  
           AAPL    234684800.0  
           ABBV      4569100.0  
           ABT       4967500.0  
...                        ...  
2023-12-29 XYL        698900.0  
           YUM       1196800.0  
           ZBH        849600.0  
           ZBRA       251300.0  
           ZTS       1007200.0  

[1229026 rows x 6 columns]

In [2]:
df['garman_klass_vol'] = ((np.log(df['high'])-np.log(df['low']))**2)/2-(2*np.log(2)-1)*((np.log(df['adj close'])-np.log(df['open']))**2)

df['rsi'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.rsi(close=x, length=20))

df['bb_low'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,0])
                                                          
df['bb_mid'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,1])
                                                          
df['bb_high'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,2])

df['sharpe_ratio'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.sharpe_ratio(close=x))

df['atr'] = df.groupby(level=1, group_keys=False).apply(compute_atr)

df['macd'] = df.groupby(level=1, group_keys=False)['adj close'].apply(compute_macd)

df['dollar_volume'] = (df['adj close']*df['volume'])/1e6

df

Price               adj close       close        high         low        open  \
date       ticker                                                               
2014-01-02 A        36.873848   40.207439   40.844063   40.164520   40.844063   
           AAL      23.907925   25.360001   25.820000   25.059999   25.070000   
           AAPL     17.273228   19.754642   19.893929   19.715000   19.845715   
           ABBV     33.865433   51.980000   52.330002   51.520000   52.119999   
           ABT      31.214535   38.230000   38.400002   38.000000   38.090000   
...                       ...         ...         ...         ...         ...   
2023-12-29 XYL     113.729843  114.360001  114.680000  113.930000  114.089996   
           YUM     129.373611  130.660004  131.250000  130.210007  130.410004   
           ZBH     121.471581  121.699997  122.400002  121.239998  121.459999   
           ZBRA    273.329987  273.329987  276.309998  272.769989  274.730011   
           ZTS     196.363464  197.369995  198.009995  196.250000  196.679993   

Price                   volume  garman_klass_vol        rsi    bb_low  \
date       ticker                                                       
2014-01-02 A         2678848.0         -0.003899        NaN       NaN   
           AAL       8997900.0         -0.000424        NaN       NaN   
           AAPL    234684800.0         -0.007405        NaN       NaN   
           ABBV      4569100.0         -0.071688        NaN       NaN   
           ABT       4967500.0         -0.015253        NaN       NaN   
...                        ...               ...        ...       ...   
2023-12-29 XYL        698900.0          0.000018  75.881265  4.647615   
           YUM       1196800.0          0.000007  59.338366  4.814819   
           ZBH        849600.0          0.000045  68.767231  4.751701   
           ZBRA       251300.0          0.000073  67.914673  5.415905   
           ZTS       1007200.0          0.000039  66.791333  5.181364   

Price                bb_mid   bb_high  sharpe_ratio       atr      macd  \
date       ticker                                                         
2014-01-02 A            NaN       NaN      0.638107       NaN       NaN   
           AAL          NaN       NaN      0.137144       NaN       NaN   
           AAPL         NaN       NaN      0.992655       NaN       NaN   
           ABBV         NaN       NaN      0.697838       NaN       NaN   
           ABT          NaN       NaN      0.652657       NaN       NaN   
...                     ...       ...           ...       ...       ...   
2023-12-29 XYL     4.702070  4.756525      0.632794 -0.071904  2.215117   
           YUM     4.852975  4.891132      0.565898  0.100551  0.893188   
           ZBH     4.782607  4.813514      0.278126 -0.907728  1.063465   
           ZBRA    5.546306  5.676707      0.614979  0.079474  1.761343   
           ZTS     5.254362  5.327361      0.861607  0.649087  2.432946   

Price              dollar_volume  
date       ticker                 
2014-01-02 A           98.779434  
           AAL        215.121115  
           AAPL      4053.763986  
           ABBV       154.734549  
           ABT        155.058201  
...                          ...  
2023-12-29 XYL         79.485787  
           YUM        154.834338  
           ZBH        103.202255  
           ZBRA        68.687826  
           ZTS        197.777281  

[1229026 rows x 15 columns]

In [3]:
data = df.groupby(level=1, group_keys=False).apply(calculate_returns).dropna()

data

Price               adj close       close        high         low        open  \
date       ticker                                                               
2014-02-07 A        38.979618   42.503578   42.539341   41.580830   41.816879   
           AAL      33.627590   35.669998   35.700001   34.650002   34.810001   
           AAPL     16.325787   18.559999   18.676071   18.477858   18.620714   
           ABBV     32.104557   48.889999   49.029999   47.660000   48.000000   
           ABT      30.526939   37.180000   37.209999   36.650002   36.770000   
...                       ...         ...         ...         ...         ...   
2023-12-29 XYL     113.729843  114.360001  114.680000  113.930000  114.089996   
           YUM     129.373611  130.660004  131.250000  130.210007  130.410004   
           ZBH     121.471581  121.699997  122.400002  121.239998  121.459999   
           ZBRA    273.329987  273.329987  276.309998  272.769989  274.730011   
           ZTS     196.363464  197.369995  198.009995  196.250000  196.679993   

Price                   volume  garman_klass_vol        rsi    bb_low  \
date       ticker                                                       
2014-02-07 A         2749586.0         -0.001647  56.320220  3.637932   
           AAL      15404700.0         -0.000016  83.448463  3.295002   
           AAPL    370280400.0         -0.006626  44.058752  2.787818   
           ABBV      6979500.0         -0.062088  46.153302  3.451851   
           ABT      12028000.0         -0.013260  46.146382  3.388760   
...                        ...               ...        ...       ...   
2023-12-29 XYL        698900.0          0.000018  75.881265  4.647615   
           YUM       1196800.0          0.000007  59.338366  4.814819   
           ZBH        849600.0          0.000045  68.767231  4.751701   
           ZBRA       251300.0          0.000073  67.914673  5.415905   
           ZTS       1007200.0          0.000039  66.791333  5.181364   

Price                bb_mid  ...  sharpe_ratio       atr      macd  \
date       ticker            ...                                     
2014-02-07 A       3.680583  ...      0.638107 -0.813500 -0.263344   
           AAL     3.427001  ...      0.137144  0.077450  2.813631   
           AAPL    2.863465  ...      0.992655 -0.929876 -0.511189   
           ABBV    3.498466  ...      0.697838 -0.934993 -0.613572   
           ABT     3.460673  ...      0.652657 -0.874355 -0.931539   
...                     ...  ...           ...       ...       ...   
2023-12-29 XYL     4.702070  ...      0.632794 -0.071904  2.215117   
           YUM     4.852975  ...      0.565898  0.100551  0.893188   
           ZBH     4.782607  ...      0.278126 -0.907728  1.063465   
           ZBRA    5.546306  ...      0.614979  0.079474  1.761343   
           ZTS     5.254362  ...      0.861607  0.649087  2.432946   

Price              dollar_volume  return_1m  return_2m  return_3m  return_6m  \
date       ticker                                                              
2014-02-07 A          107.177812   0.019386   0.017178   0.009198  -0.000252   
           AAL        518.022938   0.029141   0.027290   0.015614   0.009015   
           AAPL      6045.118789   0.013990   0.009901   0.009090   0.007530   
           ABBV       224.073756   0.020029   0.012079   0.006492   0.001991   
           ABT        367.178027   0.014461   0.011352   0.008480   0.002944   
...                          ...        ...        ...        ...        ...   
2023-12-29 XYL         79.485787   0.000350  -0.000087   0.002166   0.007004   
           YUM        154.834338   0.001073  -0.000688   0.001023   0.002082   
           ZBH        103.202255   0.000576   0.002350   0.000768   0.002680   
           ZBRA        68.687826  -0.007336  -0.004470  -0.002632   0.007331   
           ZTS        197.777281   0.001065   0.001193   0.003178   0.002333   

Price              return_9m  return_12m  
d

#### Download Fama-French Factors and Calculate Rolling Factor Betas.

We will introduce the Fama—French data to estimate the exposure of assets to common risk factors using linear regression.

The five Fama—French factors, namely market risk, size, value, operating profitability, and investment have been shown empirically to explain asset returns and are commonly used to assess the risk/return profile of portfolios. Hence, it is natural to include past factor exposures as financial features in models.

We can access the historical factor returns using the pandas-datareader and estimate historical exposures using the RollingOLS rolling linear regression.

In [4]:
factor_data = web.DataReader('F-F_Research_Data_5_Factors_2x3',
                               'famafrench',
                               start='2010')[0].drop('RF', axis=1)

factor_data.index = factor_data.index.to_timestamp()

factor_data = factor_data.resample('M').last().div(100)

factor_data.index.name = 'date'

factor_data = factor_data.join(data['return_1m']).sort_index()

factor_data

Mkt-RF     SMB     HML     RMW     CMA  return_1m
date       ticker                                                   
2014-02-28 A       0.0465  0.0014 -0.0031 -0.0023 -0.0045   0.010113
           AAL     0.0465  0.0014 -0.0031 -0.0023 -0.0045   0.010121
           AAPL    0.0465  0.0014 -0.0031 -0.0023 -0.0045  -0.002710
           ABBV    0.0465  0.0014 -0.0031 -0.0023 -0.0045   0.000000
           ABT     0.0465  0.0014 -0.0031 -0.0023 -0.0045  -0.000251
...                   ...     ...     ...     ...     ...        ...
2023-11-30 XYL     0.0884 -0.0012  0.0164 -0.0391 -0.0100   0.015749
           YUM     0.0884 -0.0012  0.0164 -0.0391 -0.0100   0.000239
           ZBH     0.0884 -0.0012  0.0164 -0.0391 -0.0100   0.020980
           ZBRA    0.0884 -0.0012  0.0164 -0.0391 -0.0100   0.001521
           ZTS     0.0884 -0.0012  0.0164 -0.0391 -0.0100   0.005006

[41032 rows x 6 columns]

In [5]:
observations = factor_data.groupby(level=1).size()

valid_stocks = observations[observations >= 10]

factor_data = factor_data[factor_data.index.get_level_values('ticker').isin(valid_stocks.index)]

factor_data

Mkt-RF     SMB     HML     RMW     CMA  return_1m
date       ticker                                                   
2014-02-28 A       0.0465  0.0014 -0.0031 -0.0023 -0.0045   0.010113
           AAL     0.0465  0.0014 -0.0031 -0.0023 -0.0045   0.010121
           AAPL    0.0465  0.0014 -0.0031 -0.0023 -0.0045  -0.002710
           ABBV    0.0465  0.0014 -0.0031 -0.0023 -0.0045   0.000000
           ABT     0.0465  0.0014 -0.0031 -0.0023 -0.0045  -0.000251
...                   ...     ...     ...     ...     ...        ...
2023-11-30 XYL     0.0884 -0.0012  0.0164 -0.0391 -0.0100   0.015749
           YUM     0.0884 -0.0012  0.0164 -0.0391 -0.0100   0.000239
           ZBH     0.0884 -0.0012  0.0164 -0.0391 -0.0100   0.020980
           ZBRA    0.0884 -0.0012  0.0164 -0.0391 -0.0100   0.001521
           ZTS     0.0884 -0.0012  0.0164 -0.0391 -0.0100   0.005006

[41017 rows x 6 columns]

Calculate Rolling Factor Betas.

In [6]:
betas = (factor_data.groupby(level=1,
                            group_keys=False)
         .apply(lambda x: RollingOLS(endog=x['return_1m'], 
                                     exog=sm.add_constant(x.drop('return_1m', axis=1)),
                                     window=min(24, x.shape[0]),
                                     min_nobs=len(x.columns)+1)
         .fit(params_only=True)
         .params
         .drop('const', axis=1)))

betas

Mkt-RF       SMB       HML       RMW       CMA
date       ticker                                                  
2014-02-28 A            NaN       NaN       NaN       NaN       NaN
           AAL          NaN       NaN       NaN       NaN       NaN
           AAPL         NaN       NaN       NaN       NaN       NaN
           ABBV         NaN       NaN       NaN       NaN       NaN
           ABT          NaN       NaN       NaN       NaN       NaN
...                     ...       ...       ...       ...       ...
2023-11-30 XYL     0.080909 -0.258150  0.118745  0.000254 -0.264882
           YUM     0.062776 -0.158095  0.142942 -0.000743 -0.159436
           ZBH     0.143147 -0.244012  0.098374 -0.174375 -0.104237
           ZBRA    0.095665 -0.109795  0.185385 -0.036818 -0.223214
           ZTS     0.052432 -0.174224  0.044772  0.022887 -0.044858

[41017 rows x 5 columns]

Join the rolling factors data to the main features dataframe

In [7]:
factors = ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']

data = (data.join(betas.groupby('ticker').shift()))

data.loc[:, factors] = data.groupby('ticker', group_keys=False)[factors].apply(lambda x: x.fillna(x.mean()))

data = data.drop('adj close', axis=1)

data = data.dropna()

data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1215623 entries, (Timestamp('2014-02-07 00:00:00'), 'A') to (Timestamp('2023-12-29 00:00:00'), 'ZTS')
Data columns (total 25 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   close             1215623 non-null  float64
 1   high              1215623 non-null  float64
 2   low               1215623 non-null  float64
 3   open              1215623 non-null  float64
 4   volume            1215623 non-null  float64
 5   garman_klass_vol  1215623 non-null  float64
 6   rsi               1215623 non-null  float64
 7   bb_low            1215623 non-null  float64
 8   bb_mid            1215623 non-null  float64
 9   bb_high           1215623 non-null  float64
 10  sharpe_ratio      1215623 non-null  float64
 11  atr               1215623 non-null  float64
 12  macd              1215623 non-null  float64
 13  dollar_volume     1215623 non-null  float64
 14  return_1m         1215623 

In [8]:
data.head()

close       high        low       open       volume  \
date       ticker                                                            
2014-02-07 A       42.503578  42.539341  41.580830  41.816879    2749586.0   
           AAL     35.669998  35.700001  34.650002  34.810001   15404700.0   
           AAPL    18.559999  18.676071  18.477858  18.620714  370280400.0   
           ABBV    48.889999  49.029999  47.660000  48.000000    6979500.0   
           ABT     37.180000  37.209999  36.650002  36.770000   12028000.0   

                   garman_klass_vol        rsi    bb_low    bb_mid   bb_high  \
date       ticker                                                              
2014-02-07 A              -0.001647  56.320220  3.637932  3.680583  3.723234   
           AAL            -0.000016  83.448463  3.295002  3.427001  3.558999   
           AAPL           -0.006626  44.058752  2.787818  2.863465  2.939113   
           ABBV           -0.062088  46.153302  3.451851  3.498466  3.545082   
           ABT            -0.013260  46.146382  3.388760  3.460673  3.532585   

                   ...  return_2m  return_3m  return_6m  return_9m  \
date       ticker  ...                                               
2014-02-07 A       ...   0.017178   0.009198  -0.000252   0.002117   
           AAL     ...   0.027290   0.015614   0.009015   0.018744   
           AAPL    ...   0.009901   0.009090   0.007530  -0.005722   
           ABBV    ...   0.012079   0.006492   0.001991   0.004795   
           ABT     ...   0.011352   0.008480   0.002944   0.002665   

                   return_12m    Mkt-RF       SMB       HML       RMW  \
date       ticker                                                       
2014-02-07 A        -0.002089 -0.017961 -0.035233  0.027932  0.013520   
           AAL       0.011220  0.002786 -0.285672  0.102337 -0.111031   
           AAPL     -0.004447  0.054401  0.005586 -0.033575  0.143428   
           ABBV     -0.000323  0.059134  0.079414 -0.098267  0.207733   
           ABT      -0.001969 -0.025529  0.059742 -0.029384  0.133738   

                        CMA  
date       ticker            
2014-02-07 A       0.028505  
           AAL    -0.018481  
           AAPL    0.196402  
           ABBV   -0.055814  
           ABT    -0.038708  

[5 rows x 25 columns]

### Save price data

In [9]:
data.reset_index(inplace = True)

In [10]:
data.to_parquet("data/price_data.gzip", compression='gzip', index = False)

In [11]:
data.head()

,date,ticker,close,high,low,open,volume,garman_klass_vol,rsi,bb_low,...,return_2m,return_3m,return_6m,return_9m,return_12m,Mkt-RF,SMB,HML,RMW,CMA
0,2014-02-07,A,42.503578,42.539341,41.580830,41.816879,2749586.0,-0.001647,56.320220,3.637932,...,0.017178,0.009198,-0.000252,0.002117,-0.002089,-0.017961,-0.035233,0.027932,0.013520,0.028505
1,2014-02-07,AAL,35.669998,35.700001,34.650002,34.810001,15404700.0,-0.000016,83.448463,3.295002,...,0.027290,0.015614,0.009015,0.018744,0.011220,0.002786,-0.285672,0.102337,-0.111031,-0.018481
2,2014-02-07,AAPL,18.559999,18.676071,18.477858,18.620714,370280400.0,-0.006626,44.058752,2.787818,...,0.009901,0.009090,0.007530,-0.005722,-0.004447,0.054401,0.005586,-0.033575,0.143428,0.196402
3,2014-02-07,ABBV,48.889999,49.029999,47.660000,48.000000,6979500.0,-0.062088,46.153302,3.451851,...,0.012079,0.006492,0.001991,0.004795,-0.000323,0.059134,0.079414,-0.098267,0.207733,-0.055814
4,2014-02-07,ABT,37.180000,37.209999,36.650002,36.770000,12028000.0,-0.013260,46.146382,3.388760,...,0.011352,0.008480,0.002944,0.002665,-0.001969,-0.025529,0.059742,-0.029384,0.133738,-0.038708


### Save data to sql db

In [19]:
import sqlite3
conn = sqlite3.connect('data/database.db')
c = conn.cursor()

In [25]:
data.to_sql('price_data', conn, if_exists='append', index = False)
master_ticker_data.to_sql('master_ticker', conn, if_exists='append')

503

### TODO: Run time series K Means clustering - DTW

### TODO: Pymc return estimation

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Sample documents
doc1 = "Machine learning is fascinating. It allows computers to learn from data."
doc2 = "Artificial intelligence, which includes machine learning, is a growing field. Computers learn from data to make decisions."

# Initialize the TF-IDF Vectorizer
vectorizer = TfidfVectorizer()

# Convert the documents into TF-IDF vectors
tfidf_matrix = vectorizer.fit_transform([doc1, doc2])

# Compute the cosine similarity
cosine_sim = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])

cosine_sim[0][0]


0.4394041187849575

In [7]:
corpus = ["I'd like an apple", 
"An apple a day keeps the doctor away", 
"Never compare an apple to an orange", 
"I prefer scikit-learn to Orange", 
"The scikit-learn docs are Orange and Blue"]                                                                                                                                                                                                   
vect = TfidfVectorizer(min_df=1, stop_words="english")                                                                                                                                                                                                   
tfidf = vect.fit_transform(corpus)                                                                                                                                                                                                                       
pairwise_similarity = tfidf * tfidf.T 
pairwise_similarity

<5x5 sparse matrix of type '<class 'numpy.float64'>'
	with 17 stored elements in Compressed Sparse Row format>

In [9]:
pairwise_similarity.toarray()

array([[1.        , 0.17668795, 0.27056873, 0.        , 0.        ],
       [0.17668795, 1.        , 0.15439436, 0.        , 0.        ],
       [0.27056873, 0.15439436, 1.        , 0.19635649, 0.16815247],
       [0.        , 0.        , 0.19635649, 1.        , 0.54499756],
       [0.        , 0.        , 0.16815247, 0.54499756, 1.        ]])